In [1]:
import numpy as np
# from tqdm import tdqm, trange

import torch
import torch.nn as nn
from torch.optim import Adam
from torch.nn import CrossEntropyLoss
from torch.utils.data import DataLoader

from torchvision.transforms import ToTensor
from torchvision.datasets.mnist import MNIST


np.random.seed(0)
torch.manual_seed(0)

In [2]:
import PIL
from PIL import Image
import numpy as np
import os
import random 
import pandas as pd
# open method used to open different extension image file
TRAIN_DATA_FOLDER = "data/train"
train_tumor_filenames = os.listdir(f'{TRAIN_DATA_FOLDER}/tumor/')
num_train_tumor_files = len(train_tumor_filenames)
print("Number of Training Tumor Files: ", num_train_tumor_files)

train_notumor_filenames = os.listdir(f'{TRAIN_DATA_FOLDER}/notumor/')
num_train_notumor_files = len(train_notumor_filenames)
print("Number of Training No Tumor Files: ", num_train_notumor_files)

Number of Training Tumor Files:  1339
Number of Training No Tumor Files:  1595


In [11]:
import os
from PIL import Image

from tqdm import tqdm, trange
import torch
from torch.utils.data import Dataset, DataLoader
import torchvision.transforms as transforms

class MRIDataset(Dataset):
    def __init__(self, root_dir, transform=None):
        self.transform = transform
        self.image_paths = []
        self.labels = []

        tumor_dir = os.path.join(root_dir, 'tumor')
        non_tumor_dir = os.path.join(root_dir, 'notumor')

        for file in os.listdir(tumor_dir):
            if file.lower().endswith('.jpg') or file.lower().endswith('.jpeg'):
                self.image_paths.append(os.path.join(tumor_dir, file))
                self.labels.append(1)

        for file in os.listdir(non_tumor_dir):
            if file.lower().endswith('.jpg') or file.lower().endswith('.jpeg'):
                self.image_paths.append(os.path.join(non_tumor_dir, file))
                self.labels.append(0)

    def __len__(self):
        return len(self.image_paths)

    def __getitem__(self, idx):
        img_path = self.image_paths[idx]
        label = self.labels[idx]
        image = Image.open(img_path).convert('RGB')
        image = image.resize(256,256)
        if self.transform:
            image = self.transform(image)
        return image, label
    
    


In [15]:
train_dataset = MRIDataset('data/train/', transform=ToTensor())
print(train_dataset)
train_loader  = DataLoader(train_dataset, batch_size=16, shuffle=True, num_workers=2)
print(train_loader)

test_dataset = MRIDataset('data/test/', transform=ToTensor())
print(test_dataset)
test_loader  = DataLoader(test_dataset, batch_size=16, shuffle=True, num_workers=2)
print(test_loader)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# print("Using device: ", device, f"({torch.cuda.get_device_name(device)})" if torch.cuda.is_available else "")
model = MyViT((1,255,255),n_patches = 7, n_blocks = 2,hidden_d = 8,n_heads = 2,out_d=10).to(device)


N_EPOCHS = 5
LR = 0.005

# Training Loop
optimizer = Adam(model.parameters(),lr = LR)
criterion = CrossEntropyLoss()
for epoch in trange(N_EPOCHS,desc="Training"):
    train_loss = 0.0
    for batch in tqdm(train_loader,desc=f"Epoch{epoch+1} in training", leave=False):
        x,y = batch
        x,y = x.to(device),y.to(device)
        y_hat = model(x)
        loss = criterion(y_hat,y)

        train_loss += loss.detach().cpu().item()/len(train_loader)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    print(f"Epoch {epoch+1}/{N_EPOCHS} loss: {train_loss:.2f}")

with torch.no_grad():
    correct,total = 0,0
    test_loss = 0.0
    for batch in tqdm(test_loader,desc="Testing"):
        x,y = batch
        x,y = x.to(device),y.to(device)
        y_hat = model(x)
        loss = criterion(x,y)
        test_loss += loss.detach().cpu().item()/len(test_loader)

        correct += torch.sum(torch.argmax(y_hat,dim=1) == y).detach().cpu()
        total += len(x)
    print(f"Test loss: {test_loss:.2f}")
    print(f"Test accuracy: {correct / total * 100:.2f}%")




NameError: name 'MyViT' is not defined

In [ ]:

train_set = MNIST(root='./../datasets', train=True, download=True, transform=ToTensor())

# test_set = MNIST(root='./../datasets', train=False, download=True, transform=transform)

